In [1]:
%matplotlib inline
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

from game import Game
from player import Player

In [2]:
from Cyganik_Chwilczynski import CyganikChwilczynski

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
### Generate cards from 9 to 14 (ace) for all colors/symbols (0, 1, 2, 3)
def getDeck():
    return [(number, color) for color in range(4) for number in range(9, 15)]
    
print(getDeck())
print(len(getDeck()))

[(9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (9, 3), (10, 3), (11, 3), (12, 3), (13, 3), (14, 3)]
24


In [5]:
### Shuffle the cards randomly. Each player gets 9 cards
### (so one player cannot be certain which cards the other player has)

def getShuffled(deck):
    D = set(deck)
    A = set(random.sample(deck, 8))
    B = set(random.sample(list(D - A), 8))
    C = D - A - B
    if len(A.intersection(B)) > 0: print("Shuffle error 1")
    if len(A.intersection(B)) > 0: print("Shuffle error 2")
    if len(A.intersection(C)) > 0: print("Shuffle error 3") 
    DS = A | B | C
    if not DS == D: print("Shuffle error 4")  
    return list(A), list(B), list(C)

p1, p2, notUsed, = getShuffled(getDeck())
print(p1)
print(p2)


[(11, 0), (14, 1), (14, 0), (13, 3), (10, 0), (12, 2), (10, 3), (14, 2)]
[(11, 1), (13, 1), (11, 3), (10, 1), (13, 0), (11, 2), (9, 1), (10, 2)]


In [6]:
# Some examplary random player

class RandomPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        ### player randomly decides which card put on the table
        card = random.choice(self.cards)
        declaration = card
        
        ### player randomly decides whether to cheat or not
        cheat = np.random.choice([True, False])
       
        ### if (s)he decides to cheat, (s)he randomly declares the card.
        if cheat:
            declaration = random.choice(self.cards)             
            
        ### Yet, declared card should be no worse than a card on the top of the pile . 
        if declared_card is not None and declaration[0] < declared_card[0]:
            declaration = (min(declared_card[0]+1,14), declaration[1])

        ### return the decision (true card) and declaration (player's declaration)
        return card, declaration
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([True, False])
    

Analyze few moves...

In [109]:
### Perform a full game 100 times
STAT_NAMES = ["Wins", "Draws", "Moves", "Cards", "Pile Size", "Checks", "Draw Decisions", "Cheats", "Errors", "Total errors"]
ANALYZE_STATS = [0, 1, 2, 3, 5, 6, 7, 8]

def printResults(results):
    print("Wins:")
    print(results[0])
    print("Draws:")
    print(results[1])
    print("Moves:")
    # print(stats_moves[2])
    print("Cards:")
    print(results[3])
    print("Pile size:")
    print(results[4])
    print("Checks:")
    print(results[5])
    print("Draw decisions:")
    print(results[6])
    print("Cheats:")
    print(results[7])
    print("Errors:")
    print(results[8])
    print("Total errors:")
    print(results[9])

def comparePlayers(player1_class, player2_class):
    stats_wins = [0, 0]
    stats_draws = [0, 0]
    stats_moves = [0, 0]
    stats_cards = [0, 0]
    stats_pile_size = 0
    stats_checks = [0, 0]
    stats_draw_decisions = [0, 0]
    stats_cheats = [0, 0]
    stats_errors = [0, 0]
    
    repeats = 1000
    errors = 0
    draws = 0

    for t in range(repeats):
        player1 = player1_class("")
        player2 = player2_class("")
        game = Game([player1, player2], log = False)
    
        error = False
        draw = False
        
        while True:
            valid, player = game.takeTurn(log = False)
            if game.moves[0] > 100 or game.moves[1] > 100:
                draws += 1
                stats_draws[0] += 1
                stats_draws[1] += 1
                if (game.player_cards[0] < game.player_cards[1]):
                    stats_wins[0] += 1
                if (game.player_cards[0] > game.player_cards[1]):
                    stats_wins[1] += 1
                    
                    
                draw=True
                # print("DRAW")
                break
            if not valid:
                error = True
                stats_errors[player] += 1
                errors += 1
                break
            if game.isFinished(log = False):
                stats_wins[player] += 1
                break
            
        stats_pile_size += len(game.pile)
        if error: continue
        #if draw: continue
       
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

    div = repeats - errors
    if div > 0:
            
        stats_pile_size /= div          
        for j in range(2):
            stats_moves[j] /= div
            stats_cheats[j] /= div
            stats_checks[j] /= div
            stats_draw_decisions[j] /= div
            stats_cards[j] /= div
            
    return [stats_wins, stats_draws, stats_moves, stats_cards, stats_pile_size, stats_checks, 
            stats_draw_decisions, stats_cheats, stats_errors, errors, draws]  


# COMPARE

In [110]:
from checker import Checker
from lier import Lier
from lier_checker import LierChecker
from more_lier import MoreLier

In [111]:
strategy = [
            ["CyganikChwilczynski", "CyganikChwilczynski", CyganikChwilczynski],
            ["RandomPlayer", "RANDOM", RandomPlayer],
            ["Lier", "Lier", Lier],
            ["Checker", "Checker", Checker],
            ["LierChecker", "LierChecker", LierChecker],
            ["MoreLier", "MoreLier", MoreLier],
           ]


In [112]:
import pandas as pd

match_results = pd.DataFrame(columns = ["Player 1", "Player 2", "Wins", "Draws", "Moves", "Cards", "Pile Size", "Checks", "Draw Decisions", "Cheats", "Errors", "Total errors"])

for A in range(len(strategy)-1):
    print("==== " + str(A), strategy[A][0])
    for B in range(A+1,len(strategy)):
        print(B, strategy[B][0])
        results = comparePlayers(strategy[A][2], strategy[B][2])
        # add a row to the dataframe
        match_results.loc[len(match_results)] = {"Player 1": strategy[A][1], "Player 2": strategy[B][1], "Wins": results[0][0], "Draws": results[1][0], "Moves": results[2][0], "Cards": results[3][0], "Pile Size": results[4], "Checks": results[5][0], "Draw Decisions": results[6][0], "Cheats": results[7][0], "Errors": results[8][0], "Total errors": results[9]}
    break

match_results

==== 0 CyganikChwilczynski
1 RandomPlayer
2 Lier
3 Checker
4 LierChecker
5 MoreLier


,Player 1,Player 2,Wins,Draws,Moves,Cards,Pile Size,Checks,Draw Decisions,Cheats,Errors,Total errors
0,CyganikChwilczynski,RANDOM,938,0,13.470,0.451,3.089,2.351,0.0,2.957,0,0
1,CyganikChwilczynski,Lier,578,0,8.996,1.707,12.026,1.053,0.0,0.675,0,0
2,CyganikChwilczynski,Checker,693,0,12.723,1.689,8.575,1.612,0.0,0.996,0,0
3,CyganikChwilczynski,LierChecker,686,0,12.971,1.766,8.607,1.823,0.0,1.020,0,0
4,CyganikChwilczynski,MoreLier,657,0,9.984,1.429,12.012,1.886,0.0,0.845,0,0
